In [1]:
# imports
import os
import pandas as pd
from src import configuration as config
from src.pipeline.pipeline_factory import PipelineFactory, ModelType, EvaluationType
from lightgbm import LGBMRegressor
from src.pipeline.pipeline_factory import PipelineFactory
from src.features.encoder_utils import load_graph
from src.pipeline.pipeline_transformers import PoincareEmbedding, ColumnKeeper


train_df = config.load_traindata_for_regression()
pipelineFactory = PipelineFactory()

graph = load_graph(config.ROOT_DIR / "data/external/graphs/encodings_graph.adjlist")


param_grid = {
    "embeddings_transformer__epochs": [500],
    "embeddings_transformer__batch_size": [10, 20, 50],
}

grid_pipeline = pipelineFactory.create_pipeline(train_df,
                                                ModelType.REGRE_BASELINE,
                                                verbose_level=1,
                                                evaluation=EvaluationType.GRID_SEARCH,
                                                param_grid=param_grid,
                                                n_folds=2,
                                                workers=8)

poincare_embedddings_transformer = PoincareEmbedding(graph=graph)
grid_pipeline.add_new_step(poincare_embedddings_transformer, "embeddings_transformer")
grid_pipeline.add_new_step(ColumnKeeper(columns=["poincare_embedding_dim1", "poincare_embedding_dim2"]), "column_keeper")
#grid_pipeline.add_new_step(PrintDataframe(verbose=grid_pipeline._verbose_level), "print_df_1")
grid_pipeline.change_estimator(LGBMRegressor())


grid_pipeline.run()


Starting pipeline using method: EvaluationType.GRID_SEARCH
Performing grid search


100%|██████████| 3/3 [01:18<00:00, 26.12s/it]

Finished running the pipeline
Evaluation metrics:
    best_score: 0.1591
    best_params: {'embeddings_transformer__epochs': 500, 'embeddings_transformer__batch_size': 20}
    results_per_parameter: [({'embeddings_transformer__epochs': 500, 'embeddings_transformer__batch_size': 10}, 0.15471857580296203), ({'embeddings_transformer__epochs': 500, 'embeddings_transformer__batch_size': 20}, 0.15908465112644707), ({'embeddings_transformer__epochs': 500, 'embeddings_transformer__batch_size': 50}, 0.15908465112644707)]
